In [2]:
import glob
import os
import os.path
import csv

# def find_all_registred_users():

all_users = (glob.glob1(os.path.abspath(os.getcwd()),"user*.txt"))
print("number of registred users:" ,len(all_users))

uuid_list = []
for i in range (len(all_users)):
    y1 = all_users[i].replace("user","")
    y2 = y1.replace(".txt","")
    uuid_list.append (y2)

token_list = ""

for i in range(len(uuid_list)):
    temp = []
    f = open("user" + uuid_list[i] + ".txt")
    contents = f.read()
    token_list = token_list + contents + "\n"
    f.close()

final_txt = open('csv.txt','w')
final_txt.write(token_list)
final_txt.close()

import csv
with open('csv.txt', 'r') as in_file:
    stripped = (line.strip() for line in in_file)
    lines = (line.split(",") for line in stripped if line)
    with open('log.csv', 'w') as out_file:
        writer = csv.writer(out_file)
        writer.writerow(('uuid', 'idtok','token'))
        writer.writerows(lines)
        

import pandas as pd
df=pd.read_csv('log.csv', sep=',', header=0)
uuids= df.values


   
    

number of registred users: 30


In [3]:
import requests as req
from authorization import authorize_user, demo_print
from helpers import send_over_air, register_two_users, retrieve_and_print_user_status, set__user_status_and_print

ERROR_SUB = "ERROR"

x = 49
y = 50


user1 = 'null'
user2 = 'null'
flagx = True
flagy = True
while flagx == True:
    for i in range (len(uuids)):
        if x == uuids[i][0]:
            print("*****")
            print("*****")
            print("*****")        
            print("User A exists: Authentication was successful")
            print("*****")
            print("*****")
            print("*****")   
            user1 = uuids[i][2]
            flagx = False
    if flagx == True: 
        print("*****")
        print("*****")
        print("User A that requested does not exist.")
        print("Logged users:")
        for z in range (len(uuids)):
            if uuids [z][0] != y:
                print(uuids [z][0])            
        x = int(input("Choose another user from list. Press 0 to quit: "))
        if x == 0: 
            exit()

while flagy == True:
    for i in range (len(uuids)):
        if y == uuids[i][0]:
            print("*****")
            print("*****")
            print("*****")        
            print("User B exists: Authentication was successful")
            print("*****")
            print("*****")
            print("*****")   
            user2 = uuids[i][2]
            flagy = False

    if flagy == True: 
        print("*****")
        print("*****")
        print("User B that requested does not exist.")
        print("Logged users:")
        for z in range (len(uuids)):
            if uuids [z][0] != x:
                print(uuids [z][0])            
        y = int(input("Choose another user from list. Press 0 to quit: "))
        if y == 0: 
            exit()


def main():
    subject = "Offline Approach"
    indent = 0
#     demo_print(subject, "starting demo", indent)
#     [user1, user2] = register_two_users()

    demo_print(subject, "retrieving status before interaction")
    retrieve_and_print_user_status(user1, user2)

    demo_print(subject, "user b gets positively diagnosed")
    set__user_status_and_print("user b", user2)

    demo_print(subject, "creating beacon keys", indent)
    demo_print("beacon 1", "generating key")
    beacon1 = generate_beacon_key()
    demo_print("beacon 2", "generating key")
    beacon2 = generate_beacon_key()

    demo_print("user a", "registering beacon to user")
    register_beacon(user1, beacon1)
    demo_print("user b", "registering beacon to user")
    register_beacon(user2, beacon2)

    demo_print("beacon 1", "transmitting token")
    beacon1_token = simulate_beacon(beacon1, 1)
    send_over_air(beacon1_token)

    demo_print("user b", "beacon token received, sending to database")
    add_interaction(user2, beacon1_token, 1)

    demo_print(subject, "retrieving status after interaction")
    retrieve_and_print_user_status(user1, user2)


def generate_beacon_key():
    demo_print("api", "retrieving factory unique generated identifier", 2)
    response = req.get("http://127.0.0.1:8080/beacon/generate")

    return response.json()["beacon_key"]


def register_beacon(auth, beacon_key):
    indent = 2
    subject = "api"
    demo_print(subject, "registering beacon: Bearer " + auth, indent)
    demo_print(subject, "beacon key: " + beacon_key, indent)
    response = req.post("http://127.0.0.1:8080/approach2/register/{}/".format(beacon_key),
                        headers={"Authorization": "Bearer " + auth})
    if response.status_code == 200:
        demo_print(subject, "Successfully added beacon", 2)
    return


def add_interaction(auth, beacon_key, timestamp):
    indent = 2
    subject = "api"
    demo_print(subject, "sending interaction to backend: Bearer " + auth, 2)
    response = req.post("http://127.0.0.1:8080/approach2/interact/{}/{}".format(beacon_key, timestamp),
                        headers={"Authorization": "Bearer " + auth})
    if response.status_code == 200:
        demo_print(subject, "successfully added interaction", indent)
    else:
        demo_print(ERROR_SUB, "API failed " + str(response.status_code), indent)
    return


def simulate_beacon(beacon_key, timestamp):
    indent = 2
    subject = "beacon"
    demo_print(subject, "beacon requesting response token at {}".format(timestamp), indent)
    demo_print(subject, "beacon key: " + beacon_key, indent)
    response = req.get("http://127.0.0.1:8080/beacon/{}/{}".format(beacon_key, timestamp))
    if response.status_code == 200:
        beacon_token = response.json()['token']
        demo_print(subject, "retrieved beacon response token: " + beacon_token, indent)
        return beacon_token
    else:
        demo_print(ERROR_SUB, "failed to create beacon transmission, error code: {}".format(response.status_code),
                   indent)


if __name__ == '__main__':
    main()


*****
*****
*****
User A exists: Authentication was successful
*****
*****
*****
*****
*****
*****
User B exists: Authentication was successful
*****
*****
*****
 [Offline Approach] retrieving status before interaction
 [user a] requesting status
   [api] retrieving user status with auth: Bearer 64b40de20a16853817792a2a0d292a8f:4f3574f48bc45a15cae41692d71b650f62b6bf3bfff99cdf01dee23e78a3fb95cd4f5bdd839b613447a837239492dff68cc3d4520c6c34494731d3892974fd0261674c55b9002550059af7f7899c9fa990ef1c68036c95eff09932704b31af05b38250a60a389c77d3da8e0987a11d626966b5f7a1613691b7e866e69d8843dffa28d50776440f6f22167da1e589e5e160001399b092606b2a6c585ffd0f0263
   [api] status retrieved
   [api] risk: 0
   [api] message: safe
 [user b] requesting status
   [api] retrieving user status with auth: Bearer 668952443122cfce86a070826d766039:3b9f17199b52a598764ff24a74a1187e23daffc6cb1956629ecb458c54408dd6643c30282b0d39ad2159af3e2e472023dce73749e15c17097eaaf49016d884b940ffe9a61a430f340c154865f1280f9f3343d4d4fe2c